# Setup

In [1]:
from datetime import datetime
import extract_msg
import os
import pandas as pd
from pdf2image import convert_from_bytes
import numpy as np
import layoutparser as lp
import pytesseract # install tesseract using windows installer here: https://github.com/UB-Mannheim/tesseract/wiki, then pip install
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\pwong\Desktop\202209_tp_deploy\TP_MP_Appeal\Tesseract-OCR\tesseract.exe'
import matplotlib.pyplot as plt
import re
# pdf2image convert_from_bytes/convert_from_path requires download poppler from https://github.com/oschwartz10612/poppler-windows/releases/
# lp.models.Detectron2LayoutModel does not work in windows. download config and model manually.
# download config here: https://layout-parser.readthedocs.io/en/latest/notes/modelzoo.html
# download models from urls stated here: https://layout-parser.readthedocs.io/en/latest/api_doc/models.html

In [2]:
INPUT_DIR = './input_data/MP Appeal (July 2021)'
OUTPUT_DIR = 'output_data'

In [3]:
def msg_to_folders(input_directory, extension='.msg', output_directory=OUTPUT_DIR):
    '''
    extract mails and attachments into folders
    '''
    num_errors=0
    for mail in os.listdir(input_directory):
        try:
            if mail.endswith(extension):
                msg = extract_msg.Message(input_directory + '/' + mail)
                msg.save(customPath=output_directory, useFileName=True)
        except Exception as e:
            num_errors+=1
            print(mail)
            print(e)
    print(f"Number of msg_to_folder extraction errors: {num_errors}")



def get_timestamp(text):
    text = datetime.strptime(str(text), '%a, %d %b %Y %H:%M:%S +0800')
    text = text.strftime('%d/%m/%Y %H:%M:%S')
    return text



def get_vehicle_number2(text):
    '''Considers the following vehicles:
    - cars
    - old cars
    - buses
    - taxis
    - goods z, y
    - goods
    - motorbikes'''
    cars = '(?<!\w)S[BCDFGJKLMNPQRSTUVWXYZ][A-Z]*\s?\d{1,4}\s?[A-Z](?!\w)'
    bikes = '(?<!\w)F[A-Z]{0,2}\s?\d{1,4}\s?[A-Z](?!\w)' #motorbikes
    bus = '(?<!\w)P[A-Z]\s?\d{1,4}\s?[A-Z](?!\w)'
    taxi = '(?<!\w)SH[A-Z]{0,1}\s?\d{1,4}\s?[A-Z](?!\w)'
    goods = '(?<!\w)G[A-Z]{0,2}\s?\d{1,4}\s?[A-Z](?!\w)'
    goods_x = '(?<!\w)X[A-Z]{0,1}\s?\d{1,4}\s?[A-Z](?!\w)'
    goods_y = '(?<!\w)TR[A-Z]{0,1}\s?\d{1,4}\s?[A-Z](?!\w)'
    trailer = '(?<!\w)TR[A-Z]{0,1}\s?\d{1,4}\s?[A-Z](?!\w)'

    veh_list = [cars, bikes, bus, taxi, goods, goods_x, goods_y, trailer]

    veh_regex = '|'.join(veh_list)
    
    try:
        veh_num = re.findall(veh_regex, text, re.I)
    except:
        veh_num = ''
        veh_num = ', '.join(set(veh_num))

    return veh_num



def msg_basic_info_extraction(input_directory, extension='.msg'):
    '''
    secondary function to extract basic data into pandas df
    '''
    num_errors=0
    my_list = []
    msg_df = pd.DataFrame()
    for mail in os.listdir(input_directory):
        try:
            if mail.endswith(extension):
                msg = extract_msg.Message(input_directory + '/' + mail)
                my_list.append([mail, msg.sender, msg.to, msg.date, msg.subject, msg.body])
                msg_df = pd.DataFrame(my_list, columns = ['msg_filename','msg_from','msg_to','msg_date','msg_subject','msg_body'])
        except Exception as e:
            num_errors+=1
            print(mail)
            print(e)
    print(f"Number of msg_basic_info extraction errors: {num_errors}")

    return msg_df



def image_to_list(image, segment_label):
    '''
    secondary function to convert image into text list
    '''
    # Segment image

    layout = model.detect(image)
    # lp.draw_box(image, layout, box_width=10) # for visualisation

    # Extract texts into a table
    
    text_blocks = lp.Layout([b for b in layout if b.type==segment_label]) # type is the segment label
    ocr_agent = lp.TesseractAgent(languages='eng')

    for block in text_blocks:
        segment_image = (block
                        .pad(left=5, right=5, top=5, bottom=5) # add padding in each image segment can help improve robustness
                        .crop_image(image))

        text = ocr_agent.detect(segment_image)
        block.set(text=text, inplace=True)

    return text_blocks



def clean_string(text_string):
    '''
    secondary function to clean a text string
    '''
    text_string = text_string.replace('\n', ' ')
    text_string = text_string.replace('\x0c', ' ')
    text_string = re.sub('\s+', ' ', text_string)
    text_string = text_string.strip()

    return text_string



def form_content_extraction(input_directory, output_directory=OUTPUT_DIR):
    '''
    extracts the form's contents into a table
    output_directory is the directory where mail contents are extracted to
    '''

    # Create a table of basic .msg info
    
    msg_df = msg_basic_info_extraction(input_directory, extension='.msg')

    # Create a table consisting of folder and file names

    os.chdir(output_directory)

    files_list = []
    for folder in os.listdir():
        os.chdir(folder)
        for files in os.listdir():
            files_list.append([folder, files])
            df = pd.DataFrame(files_list, columns=['folder', 'files'])
        os.chdir('../')

    # Merge both tables

    msg_df['msg_filename'] = msg_df['msg_filename'].str.replace('.msg$', '', regex=True)
    df_forms = msg_df.merge(df, left_on='msg_filename', right_on='folder')

    # Filter table for only pdfs known to be forms

    df_forms = df_forms[df_forms['files'].str.contains(pat='Display.aspx.pdf', regex=False) | df_forms['files'].str.contains(pat='^L.{11}.\.pdf$', regex=True)]
    # df_forms = df_forms[df_forms['files'].str.contains(pat='Display.aspx.pdf', regex=False)]
    df_forms.reset_index(drop=True, inplace=True)

    # Extract textbox data from forms into a table

    df_textboxes_appended = pd.DataFrame()
    df_freetexts_appended = []
    df_title_appended = []
    df_signoff_appended = []

    for i in range(len(df_forms)):

        # Convert pdf into numpy

        images = convert_from_bytes(pdf_file=open(f"{df_forms['folder'][i]}\{df_forms['files'][i]}", 'rb').read(),
                                   poppler_path=r'..\poppler-21.09.0\Library\bin')

        image = np.array(images[0]) # page one

        try:
            image_p2 = np.array(images[1]) # page two
        except:
            pass

        # # for visualisation
        # plt.figure(figsize=(12,16))
        # plt.imshow(image)
        # plt.axis('off')
        # plt.show()

        # Extract textboxes into a table

        text_blocks = image_to_list(image=image, segment_label=1)

        df_textboxes = pd.DataFrame(text_blocks.get_texts(), columns=[i])
        df_textboxes[i] = df_textboxes[i].str.replace('\n', ' ')
        df_textboxes[i] = df_textboxes[i].str.replace('\x0c', ' ')
        df_textboxes[i] = df_textboxes[i].str.strip()

        # Label textboxes

        textboxes_dict = {
                        df_textboxes[df_textboxes[i].str.contains(pat='Fax?No:|Fax.No:', regex=True)].index[0]: 'To / Fax No.', 
                        df_textboxes[df_textboxes[i].str.contains(pat='From:', regex=False)].index[0]: 'From / Email', 
                        df_textboxes[df_textboxes[i].str.contains(pat='Subject:', regex=False)].index[0]: 'Subject', 
                        df_textboxes[df_textboxes[i].str.contains(pat='Your?Ref:|Your.Ref:', regex=True)].index[0]: 'Your Ref / Date', 
                        df_textboxes[df_textboxes[i].str.contains(pat='Our?Ref:|Our.Ref:', regex=True)].index[0]: 'Our Ref / Date', 
                        df_textboxes[df_textboxes[i].str.contains(pat='Occupation:', regex=False)].index[0]: 'Occupation', 
                        df_textboxes[df_textboxes[i].str.contains(pat='Name:', regex=False)].index[0]: 'Name', 
                        df_textboxes[df_textboxes[i].str.contains(pat='NRIC?No\.|NRIC.No\.', regex=True)].index[0]: 'NRIC No.', 
                        df_textboxes[df_textboxes[i].str.contains(pat='Address:', regex=False)].index[0]: 'Address Plus', 
                        df_textboxes[df_textboxes[i].str.contains(pat='Marital?Status:|Marital.Status:', regex=True)].index[0]: 'Marital Status', 
                        df_textboxes[df_textboxes[i].str.contains(pat='No\.?of?Children:|No\..of.Children:|No\.?of.Children:|No\..of?Children:', regex=True)].index[0]: 'No. of Children', 
                        df_textboxes[df_textboxes[i].str.contains(pat='Date?of?Birth:|Date.of.Birth:|Date?of.Birth:|Date.of?Birth:', regex=True)].index[0]: 'DOB'
                        }

        df_textboxes = df_textboxes.rename(index=textboxes_dict)

        # Transpose textbox table

        df_textboxes_T = df_textboxes.T
        df_textboxes_T.columns = df_textboxes.index

        # Build textbox table

        df_textboxes_appended = df_textboxes_appended.append(df_textboxes_T)

        # Extract signoff into a table

        text_blocks = []
        text_blocks = image_to_list(image=image, segment_label=3)

        try:

            df_signoff = text_blocks.get_texts()[0]

            # df_signoff = clean_string(df_signoff)
            
            df_signoff_appended.append(df_signoff)

            page_two_texts = None

        except:
            
            print('Signoff extraction error')
            df_signoff_appended.append(None)

            print('Extracting page 2 texts... ')
            page_two_texts = pytesseract.image_to_string(image_p2)

        # Extract free texts into a table

        text_blocks = []
        text_blocks = image_to_list(image=image, segment_label=2)

        df_free = text_blocks.get_texts()[0]

        df_free = clean_string(df_free)
        
        if page_two_texts == None:
            df_freetexts_appended.append(df_free)
        else:
            df_freetexts_appended.append(' '.join([df_free, page_two_texts]))
            
        # Extract title into a table

        text_blocks = []
        text_blocks = image_to_list(image=image, segment_label=0)

        df_title = text_blocks.get_texts()[0]

        df_title = clean_string(df_title)
        
        df_title_appended.append(df_title)

        print(f"Completed form extraction: {i+1} out of {len(df_forms)}")

    # Join tables

    df_forms = df_forms.join(df_textboxes_appended, how='left')
    df_forms.insert(len(df_forms.columns), 'Description', df_freetexts_appended)
    df_forms.insert(len(df_forms.columns), 'Title', df_title_appended)
    df_forms.insert(len(df_forms.columns), 'Signoff', df_signoff_appended)

    return df_forms

In [4]:
# label maps ref from: https://layout-parser.readthedocs.io/en/latest/notes/modelzoo.html
# PubLayNet label map: {0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"}
# PrimaLayout label map: {1:"TextRegion", 2:"ImageRegion", 3:"TableRegion", 4:"MathsRegion", 5:"SeparatorRegion", 6:"OtherRegion"}
# TableBank label map: {0: "Table"}

model = lp.models.Detectron2LayoutModel(config_path=r'finetuned_model\config.yaml',
                                        model_path=r'finetuned_model\model_final.pth',
                                        extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8]
                                        #label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"}
                                       )

# Extract from mail

In [5]:
msg_to_folders(input_directory=INPUT_DIR, extension='.msg', output_directory=OUTPUT_DIR)

MHA Appeals Traffic Police (1).msg
Failed to create directory 'ADMIRALTY MPS: 55457A_20210727_SXXXX463B_TO_MHA-SPF'. Does it already exist?
MHA Appeals Traffic Police (15).msg
Failed to create directory 'FW: Re: Please help'. Does it already exist?
MHA Appeals Traffic Police (17).msg
Failed to create directory 'FW: MPS @ BBE on 02 JuL 2021 - Appeal letter for MR AKBARI BIN MAWI '. Does it already exist?
MHA Appeals Traffic Police (7).msg
Failed to create directory 'Re: [MPS Appeal] Appeal for waiver of traffic fine'. Does it already exist?
Number of msg_to_folder extraction errors: 4


# Extract from Forms

In [6]:
df_forms = form_content_extraction(input_directory=INPUT_DIR, output_directory=OUTPUT_DIR)


Number of msg_basic_info extraction errors: 0


c:\Users\pwong\Desktop\202209_tp_deploy\TP_MP_Appeal\venv\lib\site-packages\torch\_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ..\aten\src\ATen\native\BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
c:\Users\pwong\Desktop\202209_tp_deploy\TP_MP_Appeal\venv\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, 

Completed form extraction: 1 out of 82
Completed form extraction: 2 out of 82
Completed form extraction: 3 out of 82
Completed form extraction: 4 out of 82
Completed form extraction: 5 out of 82
Completed form extraction: 6 out of 82
Completed form extraction: 7 out of 82
Completed form extraction: 8 out of 82
Completed form extraction: 9 out of 82
Completed form extraction: 10 out of 82
Completed form extraction: 11 out of 82
Completed form extraction: 12 out of 82
Completed form extraction: 13 out of 82
Completed form extraction: 14 out of 82
Completed form extraction: 15 out of 82
Completed form extraction: 16 out of 82
Completed form extraction: 17 out of 82
Completed form extraction: 18 out of 82
Completed form extraction: 19 out of 82
Completed form extraction: 20 out of 82
Completed form extraction: 21 out of 82
Completed form extraction: 22 out of 82
Completed form extraction: 23 out of 82
Completed form extraction: 24 out of 82
Completed form extraction: 25 out of 82
Completed

In [7]:
df_forms.head(3)

,msg_filename,msg_from,msg_to,msg_date,msg_subject,msg_body,folder,files,Your Ref / Date,Our Ref / Date,...,Name,Subject,To / Fax No.,Occupation,Marital Status,From / Email,No. of Children,Description,Title,Signoff
0,Ang Mo Kio GRC MPS AH­535_210708­007­H,"""Ang Mo Kio-Hougang (PAP)"" <AMK.Hougang@pap.or...","""MHA Traffic Police (TP)"" <spf_tp_appeals@spf....","Fri, 09 Jul 2021 22:04:28 +0800",Ang Mo Kio GRC MPS AH­535/210708­007­H,Dear Sir/Madam\r\n\r\nPlease find attached. MP...,Ang Mo Kio GRC MPS AH­535_210708­007­H,Display.aspx.pdf,Your Ref: Date:,Our Ref: AH-535/210708-007-H Date: 08/07/2021,...,Name: LIM GUAN HUAT,Subject: APPEAL FOR LENIENCY,To: Fax No: COMMANDER SINGAPORE POLICE FORC...,Occupation: DELIVERY DRIVER,Marital Status: MARRIED,From: Email: DARRYL DAVID MEMBER OF PARLIAM...,No. of Children: 2,Mr Lim wishes to appeal for a waiver for two o...,PEOPLE'S ACTION PARTY ANG MO KIO-HOUGANG BRANC...,DARRYL DAVID\nANG MO KIO-HOUGANG BRANCH\nMP FO...
1,Ang Mo Kio GRC MPS AH­535_210708­007­H1,"""Ang Mo Kio-Hougang (PAP)"" <AMK.Hougang@pap.or...","""MHA Traffic Police (TP)"" <spf_tp_appeals@spf....","Fri, 09 Jul 2021 22:04:28 +0800",Ang Mo Kio GRC MPS AH­535/210708­007­H,Dear Sir/Madam\r\n\r\nPlease find attached. MP...,Ang Mo Kio GRC MPS AH­535_210708­007­H1,Display.aspx.pdf,Your Ref: Date:,Our Ref: AH-535/210708-007-H Date: 08/07/2021,...,Name: LIM GUAN HUAT,Subject: APPEAL FOR LENIENCY,To: Fax No: COMMANDER SINGAPORE POLICE FORC...,Occupation: DELIVERY DRIVER,Marital Status: MARRIED,From: Email: DARRYL DAVID MEMBER OF PARLIAM...,No. of Children: 2,Mr Lim wishes to appeal for a waiver for two o...,PEOPLE'S ACTION PARTY ANG MO KIO-HOUGANG BRANC...,DARRYL DAVID\nANG MO KIO-HOUGANG BRANCH\nMP FO...
2,Ang Mo Kio GRC MPS AH­918_210715­020­H,"""Ang Mo Kio-Hougang (PAP)"" <AMK.Hougang@pap.or...","""MHA Traffic Police (TP)"" <spf_tp_appeals@spf....","Sun, 18 Jul 2021 14:15:37 +0800",Ang Mo Kio GRC MPS AH­918/210715­020­H,Dear Sir/Madam\r\n\r\nPlease find attached. MP...,Ang Mo Kio GRC MPS AH­918_210715­020­H,Display.aspx.pdf,Your Ref: Date:,Our Ref: AH-918/210715-020-H Date: 15/07/2021,...,Name: LIM HOE PENG,Subject: APPEAL FOR LENIENCY,To: Fax No: COMMANDER SINGAPORE POLICE FORC...,Occupation: TAXI DRIVER,Marital Status: WIDOWER,From: Email: DARRYL DAVID MEMBER OF PARLIAM...,No. of Children: 2,Mr Lim wishes to appeal for leniency for his t...,PEOPLE'S ACTION PARTY ANG MO KIO-HOUGANG BRANC...,DARRYL DAVID\nANG MO KIO-HOUGANG BRANCH\nMP FO...


In [8]:
df_forms.to_csv('raw_table.csv', index=False, mode='x')

In [9]:
# df_forms.drop_duplicates(subset=['msg_from', 'msg_date', 'msg_subject'], inplace=True)

# Process relevant columns
1. Time Stamp
2. Volunteers' Email Address
3. (ELITES/TIMS) - ignore
4. MP Ward/GRC
5. Member of Parliament
6. TP No.
7. Report No.
8. Name
9. NRIC
10. Reason for Appeal
11. Details for Appeal
12. Mobile No.
13. House No.
14. Email Address (Driver) - ignore
15. Email Address
16. Vehicle No.

In [10]:
# alerts to highlight for any errors detected

df_forms['alerts_nric'] = None
df_forms['alerts_report_no'] = None

# get volunteers' email address

v_email = []

for i in range(len(df_forms)):
    try:
        address = re.search('<.*>', df_forms['msg_from'][i]).group(0)
        address = re.sub('\s+', '', address)
    except:
        address = None
    v_email.append(address)

df_forms.insert(len(df_forms.columns), 'Volunteers\' Email Address', v_email)

df_forms['Volunteers\' Email Address'] = df_forms['Volunteers\' Email Address'].str.replace('<', '', regex=False)
df_forms['Volunteers\' Email Address'] = df_forms['Volunteers\' Email Address'].str.replace('>', '', regex=False)
df_forms['Volunteers\' Email Address'] = df_forms['Volunteers\' Email Address'].str.strip()

# get timestamp

timestamp = []

for i in range(len(df_forms)):

    date_time = get_timestamp(df_forms['msg_date'][i])
    timestamp.append(date_time)

df_forms.insert(len(df_forms.columns), 'Time Stamp', timestamp)

# get MP name, MP Ward/GRC

df_forms['From / Email'] = df_forms['From / Email'].str.replace('From:', '', regex=False)
df_forms['From / Email'] = df_forms['From / Email'].str.replace('Email:', '', regex=False)
df_forms['From / Email'] = df_forms['From / Email'].str.strip()

mp = []

for i in range(len(df_forms)):

    try:
        if df_forms['From / Email'][i].find('MEMBER') != -1:
            mp.append(df_forms['From / Email'][i].partition('MEMBER')[0])
        else:
            mp.append(df_forms['From / Email'][i].partition('MP')[0])
    except:
        mp.append(None)

df_forms.insert(len(df_forms.columns), 'Member of Parliament', mp)
df_forms['Member of Parliament'] = df_forms['Member of Parliament'].str.upper()
df_forms['Member of Parliament'] = df_forms['Member of Parliament'].str.replace('POH LISAN', 'POH LI SAN', regex=False)
df_forms['Member of Parliament'] = df_forms['Member of Parliament'].str.strip()

mp_ward_grc = []

for i in range(len(df_forms)):

    try:
        if df_forms['From / Email'][i].find('MEMBER') != -1:
            mp_ward_grc.append(re.search('PARLIAMENT.*GRC|PARLIAMENT.*SMC', df_forms['From / Email'][i]).group(0))
        else:
            mp_ward_grc.append(re.search('MP.*GRC|MP.*SMC', df_forms['From / Email'][i]).group(0))
    except:
        mp_ward_grc.append(None)

df_forms.insert(len(df_forms.columns), 'MP Ward/GRC', mp_ward_grc)

df_forms['MP Ward/GRC'] = df_forms['MP Ward/GRC'].str.replace('PARLIAMENT', '', regex=False)
df_forms['MP Ward/GRC'] = df_forms['MP Ward/GRC'].str.replace('MP FOR', '', regex=False)
df_forms['MP Ward/GRC'] = df_forms['MP Ward/GRC'].str.replace('MP  FOR', '', regex=False)
df_forms['MP Ward/GRC'] = df_forms['MP Ward/GRC'].str.replace('MP OF', '', regex=False)
df_forms['MP Ward/GRC'] = df_forms['MP Ward/GRC'].str.replace('FOR', '', regex=False)
df_forms['MP Ward/GRC'] = df_forms['MP Ward/GRC'].str.replace('\s+', ' ', regex=True)
df_forms['MP Ward/GRC'] = df_forms['MP Ward/GRC'].str.replace('CHENG SAN-SELETAR ANG MO KIO GRC', 'ANG MO KIO GRC', regex=False)
df_forms['MP Ward/GRC'] = df_forms['MP Ward/GRC'].str.strip()

# get rpt no.

rpt_no = []

for i in range(len(df_forms)):
    rpt_no.append(df_forms['Your Ref / Date'][i].partition('Date:')[0])

for i in range(len(rpt_no)):
    try:
        rpt_no[i] = re.sub('\s+', '', rpt_no[i])
        rpt_no[i] = re.search('\d{12}', rpt_no[i]).group(0)
    except:
        rpt_no[i] = None

df_forms.insert(len(df_forms.columns), 'Report No.', rpt_no)

# get tp no.

tp_no = []

for i in range(len(df_forms)):
    tp_no.append(df_forms['Your Ref / Date'][i].partition('Date:')[0])

for i in range(len(tp_no)):
    try:
        tp_no[i] = re.sub('\s+', '', tp_no[i])
        tp_no[i] = tp_no[i].upper()
        tp_no[i] = re.search('TP/IP/\d{5}/\d{4}', tp_no[i]).group(0)
    except:
        tp_no[i] = None

df_forms.insert(len(df_forms.columns), 'TP No.', tp_no)

# get name

df_forms['Name'] = df_forms['Name'].str.replace('Name:', '', regex=False)
df_forms['Name'] = df_forms['Name'].str.replace(',$', '', regex=True)
df_forms['Name'] = df_forms['Name'].str.replace('\$', 'S', regex=True)
df_forms['Name'] = df_forms['Name'].str.upper()
df_forms['Name'] = df_forms['Name'].str.strip()

# get NRIC # no solution for last char, I or J, being mis-recognised as ), ], 1? 

df_forms['NRIC No.'] = df_forms['NRIC No.'].str.replace('NRIC No.', '', regex=False)
df_forms['NRIC No.'] = df_forms['NRIC No.'].str.replace('NRICNo.', '', regex=False)
df_forms['NRIC No.'] = df_forms['NRIC No.'].str.replace('\$', 'S', regex=True)
df_forms['NRIC No.'] = df_forms['NRIC No.'].str.replace('(Pink)', '', regex=False)
df_forms['NRIC No.'] = df_forms['NRIC No.'].str.replace('(Blue)', '', regex=False)
df_forms['NRIC No.'] = df_forms['NRIC No.'].str.replace('\s+', '', regex=True)
df_forms['NRIC No.'] = df_forms['NRIC No.'].str.upper()
df_forms['NRIC No.'] = df_forms['NRIC No.'].str.strip()

for i in range(len(df_forms)):
    if re.search('^[A-Z]\d{7}[A-Z]$', df_forms['NRIC No.'][i]):
        continue
    else:
        # df_forms['NRIC No.'][i] = None
        df_forms['alerts_nric'][i] = 'Error in NRIC detected'

# get Reason for Appeal

df_forms['Subject'] = df_forms['Subject'].str.replace('Subject:', '', regex=False)
df_forms['Subject'] = df_forms['Subject'].str.replace('\|', '', regex=True)
df_forms['Subject'] = df_forms['Subject'].str.replace(':', '', regex=False)
df_forms['Subject'] = df_forms['Subject'].str.replace('\s+', ' ', regex=True)
df_forms['Subject'] = df_forms['Subject'].str.strip()

# get Mobile No., House No., Email Address

df_forms['Address Plus'] = df_forms['Address Plus'].str.replace('Address', '', regex=False)
df_forms['Address Plus'] = df_forms['Address Plus'].str.replace('Tel', '', regex=False)
df_forms['Address Plus'] = df_forms['Address Plus'].str.replace('Email', '', regex=False)
df_forms['Address Plus'] = df_forms['Address Plus'].str.strip()

mobile_no = []
house_no = []
email = []

for i in range(len(df_forms)):
    try:
        mobile_no.append(df_forms['Address Plus'][i].split(':')[2])
    except:
        mobile_no.append(None)

for i in range(len(df_forms)):
    try:
        house_no.append(df_forms['Address Plus'][i].split(':')[1])
    except:
        house_no.append(None)

for i in range(len(df_forms)):
    try:
        address = df_forms['Address Plus'][i].split(':')[3]
        address = re.search('[\S\w]+@[\S\w]+', address).group(0)
        email.append(address)
    except:
        email.append(None)

df_forms.insert(len(df_forms.columns), 'Mobile No.', mobile_no)
df_forms.insert(len(df_forms.columns), 'House No.', house_no)
df_forms.insert(len(df_forms.columns), 'Email Address', email)

df_forms['Mobile No.'] = df_forms['Mobile No.'].str.replace('\s+', '', regex=True)
df_forms['Mobile No.'] = df_forms['Mobile No.'].str.replace(',', '', regex=False)
df_forms['Mobile No.'] = df_forms['Mobile No.'].str.strip()
df_forms['House No.'] = df_forms['House No.'] .str.replace('\|', '1', regex=True)
df_forms['House No.'] = df_forms['House No.'].str.strip()
df_forms['Email Address'] = df_forms['Email Address'].str.replace('—', '', regex=False)
df_forms['Email Address'] = df_forms['Email Address'].str.replace('^_', '', regex=True)
df_forms['Email Address'] = df_forms['Email Address'].str.strip()

df_forms.head(3)

,msg_filename,msg_from,msg_to,msg_date,msg_subject,msg_body,folder,files,Your Ref / Date,Our Ref / Date,...,alerts_report_no,Volunteers' Email Address,Time Stamp,Member of Parliament,MP Ward/GRC,Report No.,TP No.,Mobile No.,House No.,Email Address
0,Ang Mo Kio GRC MPS AH­535_210708­007­H,"""Ang Mo Kio-Hougang (PAP)"" <AMK.Hougang@pap.or...","""MHA Traffic Police (TP)"" <spf_tp_appeals@spf....","Fri, 09 Jul 2021 22:04:28 +0800",Ang Mo Kio GRC MPS AH­535/210708­007­H,Dear Sir/Madam\r\n\r\nPlease find attached. MP...,Ang Mo Kio GRC MPS AH­535_210708­007­H,Display.aspx.pdf,Your Ref: Date:,Our Ref: AH-535/210708-007-H Date: 08/07/2021,...,None,AMK.Hougang@pap.org.sg,09/07/2021 22:04:28,DARRYL DAVID,ANG MO KIO GRC,None,None,97961113,535 HOUGANG STREET 52 #03-22 SINGAPORE 530535,None
1,Ang Mo Kio GRC MPS AH­535_210708­007­H1,"""Ang Mo Kio-Hougang (PAP)"" <AMK.Hougang@pap.or...","""MHA Traffic Police (TP)"" <spf_tp_appeals@spf....","Fri, 09 Jul 2021 22:04:28 +0800",Ang Mo Kio GRC MPS AH­535/210708­007­H,Dear Sir/Madam\r\n\r\nPlease find attached. MP...,Ang Mo Kio GRC MPS AH­535_210708­007­H1,Display.aspx.pdf,Your Ref: Date:,Our Ref: AH-535/210708-007-H Date: 08/07/2021,...,None,AMK.Hougang@pap.org.sg,09/07/2021 22:04:28,DARRYL DAVID,ANG MO KIO GRC,None,None,97961113,535 HOUGANG STREET 52 #03-22 SINGAPORE 530535,None
2,Ang Mo Kio GRC MPS AH­918_210715­020­H,"""Ang Mo Kio-Hougang (PAP)"" <AMK.Hougang@pap.or...","""MHA Traffic Police (TP)"" <spf_tp_appeals@spf....","Sun, 18 Jul 2021 14:15:37 +0800",Ang Mo Kio GRC MPS AH­918/210715­020­H,Dear Sir/Madam\r\n\r\nPlease find attached. MP...,Ang Mo Kio GRC MPS AH­918_210715­020­H,Display.aspx.pdf,Your Ref: Date:,Our Ref: AH-918/210715-020-H Date: 15/07/2021,...,None,AMK.Hougang@pap.org.sg,18/07/2021 14:15:37,DARRYL DAVID,ANG MO KIO GRC,None,None,90061988,918 HOUGANG AVENUE 9 #04-42 SINGAPORE 530918,None


# Layer 1 Checkpoint
Layer 1 comprises extraction from 1. msg datetime and sender; and 2. form textboxes

In [11]:
mastersheet = df_forms[['folder',
                        'files',
                        'Time Stamp', 
                        'Volunteers\' Email Address', 
                        'MP Ward/GRC', 
                        'Member of Parliament', 
                        'TP No.', 
                        'Report No.', 
                        'Name', 
                        'NRIC No.', 
                        'Subject', 
                        'Description', 
                        'Mobile No.', 
                        'House No.',
                        'Email Address',
                        'alerts_nric',
                        'alerts_report_no',
                        'Signoff']]
mastersheet.columns = ['folder',
                        'files',
                        'Time Stamp', 
                        'Volunteers\' Email Address', 
                        'MP Ward/GRC', 
                        'Member of Parliament', 
                        'TP No.', 
                        'Report No.', 
                        'Name', 
                        'NRIC', 
                        'Reason for Appeal', 
                        'Details for Appeal', 
                        'Mobile No.', 
                        'House No.',
                        'Email Address',
                        'alerts_nric',
                        'alerts_report_no',
                        'Signoff']

mastersheet.head(3)

,folder,files,Time Stamp,Volunteers' Email Address,MP Ward/GRC,Member of Parliament,TP No.,Report No.,Name,NRIC,Reason for Appeal,Details for Appeal,Mobile No.,House No.,Email Address,alerts_nric,alerts_report_no,Signoff
0,Ang Mo Kio GRC MPS AH­535_210708­007­H,Display.aspx.pdf,09/07/2021 22:04:28,AMK.Hougang@pap.org.sg,ANG MO KIO GRC,DARRYL DAVID,None,None,LIM GUAN HUAT,S1537456Z,APPEAL FOR LENIENCY,Mr Lim wishes to appeal for a waiver for two o...,97961113,535 HOUGANG STREET 52 #03-22 SINGAPORE 530535,None,None,None,DARRYL DAVID\nANG MO KIO-HOUGANG BRANCH\nMP FO...
1,Ang Mo Kio GRC MPS AH­535_210708­007­H1,Display.aspx.pdf,09/07/2021 22:04:28,AMK.Hougang@pap.org.sg,ANG MO KIO GRC,DARRYL DAVID,None,None,LIM GUAN HUAT,S1537456Z,APPEAL FOR LENIENCY,Mr Lim wishes to appeal for a waiver for two o...,97961113,535 HOUGANG STREET 52 #03-22 SINGAPORE 530535,None,None,None,DARRYL DAVID\nANG MO KIO-HOUGANG BRANCH\nMP FO...
2,Ang Mo Kio GRC MPS AH­918_210715­020­H,Display.aspx.pdf,18/07/2021 14:15:37,AMK.Hougang@pap.org.sg,ANG MO KIO GRC,DARRYL DAVID,None,None,LIM HOE PENG,S0028533A,APPEAL FOR LENIENCY,Mr Lim wishes to appeal for leniency for his t...,90061988,918 HOUGANG AVENUE 9 #04-42 SINGAPORE 530918,None,None,None,DARRYL DAVID\nANG MO KIO-HOUGANG BRANCH\nMP FO...


In [12]:
mastersheet.to_csv('stage1_checkpoint.csv', index=False, mode='x')

# Layer 2 Checkpoint

In [13]:
mastersheet['Details for Appeal'] = mastersheet['Details for Appeal'].str.replace('([0-9])\s([0-9])', r'\1\2', regex=True)

C:\Users\pwong\AppData\Local\Temp/ipykernel_11520/1824537171.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mastersheet['Details for Appeal'] = mastersheet['Details for Appeal'].str.replace('([0-9])\s([0-9])', r'\1\2', regex=True)


In [14]:
# tp no.

for i in range(len(mastersheet)):

    try:
        if mastersheet['TP No.'][i] == None:
            if len(re.findall('TP/IP/\d{5}/\d{4}', mastersheet['Details for Appeal'][i])) == 0:
                mastersheet['TP No.'][i] = None
            elif len(re.findall('TP/IP/\d{5}/\d{4}', mastersheet['Details for Appeal'][i])) == 1:
                mastersheet['TP No.'][i] = re.findall('TP/IP/\d{5}/\d{4}', mastersheet['Details for Appeal'][i])[0]
            else:
                mastersheet['TP No.'][i] = re.findall('TP/IP/\d{5}/\d{4}', mastersheet['Details for Appeal'][i])
    except:
        mastersheet['TP No.'][i] = None

C:\Users\pwong\AppData\Local\Temp/ipykernel_11520/1720684503.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mastersheet['TP No.'][i] = None
C:\Users\pwong\AppData\Local\Temp/ipykernel_11520/1720684503.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mastersheet['TP No.'][i] = re.findall('TP/IP/\d{5}/\d{4}', mastersheet['Details for Appeal'][i])[0]


In [15]:
# rpt no.

for i in range(len(mastersheet)):

    try:
        if mastersheet['Report No.'][i] == None:
            if len(re.findall('\d{4}[\s,-]?\d{4}[\s,-]?\d{4}', mastersheet['Details for Appeal'][i])) == 0:
                mastersheet['Report No.'][i] = None
            elif len(re.findall('\d{4}[\s,-]?\d{4}[\s,-]?\d{4}', mastersheet['Details for Appeal'][i])) == 1:
                mastersheet['Report No.'][i] = re.findall('\d{4}[\s,-]?\d{4}[\s,-]?\d{4}', mastersheet['Details for Appeal'][i])[0]
            else:
                mastersheet['Report No.'][i] = re.findall('\d{4}[\s,-]?\d{4}[\s,-]?\d{4}', mastersheet['Details for Appeal'][i])
    except:
        mastersheet['Report No.'][i] = None

for i in range(len(mastersheet)):

    if mastersheet['Report No.'][i] == None:
        continue

    elif len(mastersheet['Report No.'][i]) == 1:
        
        if bool(re.match('\d{4}[\s,-]?\d{4}[\s,-]?\d{4}', mastersheet['Report No.'][i])): # incorrect
            continue
        else:
            mastersheet['alerts_report_no'][i] = 'Error in Report No. detected'

        continue
    
    else:

        for j in range(len(mastersheet['Report No.'][i])):

            if bool(re.match('\d{4}[\s,-]?\d{4}[\s,-]?\d{4}', mastersheet['Report No.'][i][j])):
                continue
            else:
                mastersheet['alerts_report_no'][i] = 'Error in Report No. detected'

C:\Users\pwong\AppData\Local\Temp/ipykernel_11520/1942111081.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mastersheet['Report No.'][i] = None
C:\Users\pwong\AppData\Local\Temp/ipykernel_11520/1942111081.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mastersheet['Report No.'][i] = re.findall('\d{4}[\s,-]?\d{4}[\s,-]?\d{4}', mastersheet['Details for Appeal'][i])
C:\Users\pwong\AppData\Local\Temp/ipykernel_11520/1942111081.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

In [16]:
# email address

for i in range(len(mastersheet)):

    try:
        if mastersheet['Email Address'][i] == None:
            if len(re.findall('[\S\w]+@[\S\w]+', mastersheet['Details for Appeal'][i])) == 0:
                mastersheet['Email Address'][i] = None
            elif len(re.findall('[\S\w]+@[\S\w]+', mastersheet['Details for Appeal'][i])) == 1:
                mastersheet['Email Address'][i] = re.findall('[\S\w]+@[\S\w]+', mastersheet['Details for Appeal'][i])[0]
            else:
                mastersheet['Email Address'][i] = re.findall('[\S\w]+@[\S\w]+', mastersheet['Details for Appeal'][i])
    except:
        mastersheet['Email Address'][i] = None

C:\Users\pwong\AppData\Local\Temp/ipykernel_11520/2532858887.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mastersheet['Email Address'][i] = None


In [17]:
# veh. no.

mastersheet['Veh No.'] = None

for i in range(len(mastersheet)):

    try:
        if mastersheet['Veh No.'][i] == None:
            if len(get_vehicle_number2(mastersheet['Details for Appeal'][i])) == 0:
                mastersheet['Veh No.'][i] = None
            elif len(get_vehicle_number2(mastersheet['Details for Appeal'][i])) == 1:
                mastersheet['Veh No.'][i] = get_vehicle_number2(mastersheet['Details for Appeal'][i])[0]
            else:
                # mastersheet['Veh No.'][i] = get_vehicle_number2(mastersheet['Details for Appeal'][i])
                veh_no_list = get_vehicle_number2(mastersheet['Details for Appeal'][i])
                new_veh_no_list = []
                for j in range(len(veh_no_list)):
                    if veh_no_list[j].isupper():
                        new_veh_no_list.append(veh_no_list[j])
                if len(new_veh_no_list) == 0:
                    mastersheet['Veh No.'][i] = None
                else:
                    mastersheet['Veh No.'][i] = new_veh_no_list

    except:
        mastersheet['Veh No.'][i] = None

C:\Users\pwong\AppData\Local\Temp/ipykernel_11520/1547659063.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mastersheet['Veh No.'] = None
C:\Users\pwong\AppData\Local\Temp/ipykernel_11520/1547659063.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mastersheet['Veh No.'][i] = None
C:\Users\pwong\AppData\Local\Temp/ipykernel_11520/1547659063.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [18]:
# mp ward / grc

for i in range(len(mastersheet)):
    if mastersheet['MP Ward/GRC'][i] == None:
        try:
            if 'FOR' in mastersheet['Signoff'][i]:
                mastersheet['MP Ward/GRC'][i] = mastersheet['Signoff'][i].split('FOR ')[-1]
                mastersheet['MP Ward/GRC'][i] = mastersheet['MP Ward/GRC'][i].replace('\n', '')
            else:
                mastersheet['MP Ward/GRC'][i] = mastersheet['Signoff'][i].split(', ')[-1]
                mastersheet['MP Ward/GRC'][i] = mastersheet['MP Ward/GRC'][i].replace('\n', '')
        except:
            continue

mastersheet.drop('Signoff', axis=1, inplace=True)

C:\Users\pwong\AppData\Local\Temp/ipykernel_11520/1818356496.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mastersheet['MP Ward/GRC'][i] = mastersheet['Signoff'][i].split(', ')[-1]
C:\Users\pwong\AppData\Local\Temp/ipykernel_11520/1818356496.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mastersheet['MP Ward/GRC'][i] = mastersheet['MP Ward/GRC'][i].replace('\n', '')
c:\Users\pwong\Desktop\202209_tp_deploy\TP_MP_Appeal\venv\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pan

In [19]:
mastersheet.to_csv('stage2_checkpoint.csv', index=False, mode='x')

In [20]:
# import shutil
# import os

# source = 'c:\\Users\\pwong\\Desktop\\TP_MP_Appeal\\output_data'
# destination = 'c:\\Users\\pwong\\Desktop\\TP_MP_Appeal\\collect_pdf'

# for root, dirs, files in os.walk((os.path.normpath(source)), topdown=False):
#     for name in files:
#         if name.endswith(('.pdf', '.PDF')):
#             SourceFolder = os.path.join(root, name)
#             shutil.copy2(SourceFolder, destination)